In [230]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

In [231]:
df_train = pd.read_csv('../data/train_cleaned.csv')
df_test = pd.read_csv('../data/test_cleaned.csv')

df_train.head()

,Unnamed: 0.1,Unnamed: 0,title,address,property_name,property_type,num_beds,num_baths,size_sqft,furnishing,lat,lng,subzone,planning_area,price,built_year,tenure,available_unit_types
0,0,0,hdb flat for sale in 506b serangoon north aven...,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,4,2,1575,unspecified,1.373047,103.8750,serangoon north,serangoon,995400.0,1992,99-year leasehold,"1, 2, 3, 4, 5, 6 br"
1,1,1,4 bed condo for sale in meyerhouse,128 meyer road,meyerhouse,condo,4,6,3070,partial,1.298828,103.8750,mountbatten,marine parade,8485000.0,2022,freehold,"studio, 3, 4, 5, 6 br"
2,2,2,3 bed condo for sale in leedon green,26 leedon heights,leedon green,condo,3,2,958,partial,1.312500,103.8125,farrer court,bukit timah,2626000.0,2023,freehold,"studio, 1, 2, 3, 4 br"
3,3,3,2 bed condo for sale in one bernam,1 bernam street,one bernam,condo,2,1,732,unspecified,1.274414,103.8125,anson,downtown core,1764000.0,2026,99-year leasehold,"studio, 1, 2, 3, 4, 5 br"
4,4,4,2 bed condo for sale in forett @ bukit timah,32 toh tuck road,forett @ bukit timah,condo,2,2,689,unspecified,1.338867,103.7500,bukit batok south,bukit batok,1702000.0,2024,freehold,"studio, 1, 2, 3, 4, 5 br"


In [232]:
df_train = df_train.drop(['Unnamed: 0.1', 'Unnamed: 0', 'title'], axis=1)
df_train.head()

,address,property_name,property_type,num_beds,num_baths,size_sqft,furnishing,lat,lng,subzone,planning_area,price,built_year,tenure,available_unit_types
0,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,4,2,1575,unspecified,1.373047,103.8750,serangoon north,serangoon,995400.0,1992,99-year leasehold,"1, 2, 3, 4, 5, 6 br"
1,128 meyer road,meyerhouse,condo,4,6,3070,partial,1.298828,103.8750,mountbatten,marine parade,8485000.0,2022,freehold,"studio, 3, 4, 5, 6 br"
2,26 leedon heights,leedon green,condo,3,2,958,partial,1.312500,103.8125,farrer court,bukit timah,2626000.0,2023,freehold,"studio, 1, 2, 3, 4 br"
3,1 bernam street,one bernam,condo,2,1,732,unspecified,1.274414,103.8125,anson,downtown core,1764000.0,2026,99-year leasehold,"studio, 1, 2, 3, 4, 5 br"
4,32 toh tuck road,forett @ bukit timah,condo,2,2,689,unspecified,1.338867,103.7500,bukit batok south,bukit batok,1702000.0,2024,freehold,"studio, 1, 2, 3, 4, 5 br"


In [233]:
#Calculate price per sqft of every row
df_train["price_per_sqft"] = df_train["price"]/df_train["size_sqft"] 
df_train["price_per_sqft"] = df_train["price_per_sqft"].round(2)


In [234]:
#Target encode Property Name
prop_name_agg = df_train.groupby('property_name').agg('mean')
df_train = df_train.merge(prop_name_agg["price_per_sqft"], on=['property_name'], how='left')
df_train.rename(columns={"price_per_sqft_y": "property_name_encoded"}, inplace=True)
df_train.drop(["property_name", "available_unit_types"], axis=1, inplace=True)
df_train.head()

#TODO: Apply to test data


,address,property_type,num_beds,num_baths,size_sqft,furnishing,lat,lng,subzone,planning_area,price,built_year,tenure,price_per_sqft_x,property_name_encoded
0,hougang / punggol / sengkang (d19),hdb,4,2,1575,unspecified,1.373047,103.8750,serangoon north,serangoon,995400.0,1992,99-year leasehold,632.00,811.803488
1,128 meyer road,condo,4,6,3070,partial,1.298828,103.8750,mountbatten,marine parade,8485000.0,2022,freehold,2763.84,2812.763529
2,26 leedon heights,condo,3,2,958,partial,1.312500,103.8125,farrer court,bukit timah,2626000.0,2023,freehold,2741.13,2946.238404
3,1 bernam street,condo,2,1,732,unspecified,1.274414,103.8125,anson,downtown core,1764000.0,2026,99-year leasehold,2409.84,2880.462661
4,32 toh tuck road,condo,2,2,689,unspecified,1.338867,103.7500,bukit batok south,bukit batok,1702000.0,2024,freehold,2470.25,2238.394476


In [235]:
#Target encode Address

address_agg = df_train.groupby('address').agg('mean')
df_train = df_train.merge(address_agg["price_per_sqft_x"], on=['address'], how='left')
df_train.rename(columns={"price_per_sqft_x_x": "price_per_sqft"}, inplace=True)
df_train.rename(columns={"price_per_sqft_x_y": "address_encoded"}, inplace=True)
df_train.drop(["address"], axis=1, inplace=True)

#TODO: Apply to test data



In [236]:
#Target encode Subzone and drop planning Area
subzone_agg = df_train.groupby('subzone').agg('mean')
df_train = df_train.merge(subzone_agg["price_per_sqft"], on=['subzone'], how='left')
df_train.rename(columns={"price_per_sqft_x": "price_per_sqft"}, inplace=True)
df_train.rename(columns={"price_per_sqft_y": "subzone_encoded"}, inplace=True)
df_train.drop(["subzone", "planning_area"], axis=1, inplace=True)

#TODO: Apply to test data


In [237]:
#Label Encode property_type
pd.set_option("display.max_columns", 15)
type_to_enum_map = {t: idx+1 for idx, t in enumerate(sorted(df_train.property_type.unique()))}
df_train.property_type = df_train.property_type.replace(type_to_enum_map,).astype(np.int16)
df_train.head()

df_test.property_type = df_test.property_type.replace(type_to_enum_map,).astype(np.int16)


In [238]:
#One hot encode furnishing
x = pd.get_dummies(df_train.furnishing)
df_train['furnishing_unspecified'] = x.unspecified.astype(int)
df_train['furnishing_partial'] = x.partial.astype(int)
df_train['furnishing_unfurnished'] = x.unfurnished.astype(int)
df_train['furnishing_fully'] = x.fully.astype(int)
df_train.drop(columns=['furnishing'], axis = 1, inplace=True)

#TODO: Apply to test data


df_train.head()


,property_type,num_beds,num_baths,size_sqft,lat,lng,price,...,property_name_encoded,address_encoded,subzone_encoded,furnishing_unspecified,furnishing_partial,furnishing_unfurnished,furnishing_fully
0,8,4,2,1575,1.373047,103.8750,995400.0,...,811.803488,248880.379809,626.643333,1,0,0,0
1,4,4,6,3070,1.298828,103.8750,8485000.0,...,2812.763529,2812.763529,2474.825711,0,1,0,0
2,4,3,2,958,1.312500,103.8125,2626000.0,...,2946.238404,2944.604516,2864.480273,0,1,0,0
3,4,2,1,732,1.274414,103.8125,1764000.0,...,2880.462661,2880.462661,2845.122957,1,0,0,0
4,4,2,2,689,1.338867,103.7500,1702000.0,...,2238.394476,2262.348361,1808.754700,1,0,0,0


In [239]:
#One hot encode tenure

pd.set_option("display.max_columns", 25)

x = pd.get_dummies(df_train.tenure)
#print(x)
df_train['tenure_99'] = x['99-year leasehold'].astype(int)
df_train['tenure_999'] = x['999-year leasehold'].astype(int)
df_train['tenure_freehold'] = x['freehold'].astype(int)
df_train.drop(columns=['tenure', 'lat', 'lng'], axis = 1, inplace=True)

#TODO: Apply to test data


In [240]:
#Built Year
df_train.built_year = 2022 - df_train.built_year
df_test.built_year = 2022 - df_test.built_year

df_train.head()


,property_type,num_beds,num_baths,size_sqft,price,built_year,price_per_sqft,property_name_encoded,address_encoded,subzone_encoded,furnishing_unspecified,furnishing_partial,furnishing_unfurnished,furnishing_fully,tenure_99,tenure_999,tenure_freehold
0,8,4,2,1575,995400.0,30,632.00,811.803488,248880.379809,626.643333,1,0,0,0,1,0,0
1,4,4,6,3070,8485000.0,0,2763.84,2812.763529,2812.763529,2474.825711,0,1,0,0,0,0,1
2,4,3,2,958,2626000.0,-1,2741.13,2946.238404,2944.604516,2864.480273,0,1,0,0,0,0,1
3,4,2,1,732,1764000.0,-4,2409.84,2880.462661,2880.462661,2845.122957,1,0,0,0,1,0,0
4,4,2,2,689,1702000.0,-2,2470.25,2238.394476,2262.348361,1808.754700,1,0,0,0,0,0,1


In [241]:
df_train.to_csv('../data/train_encoded.csv')
df_test.to_csv('../data/test_encoded.csv')